In [3]:
path_dataset = 'data/ganTrialElectrodeERP_p50_e8_len100.csv'
save_name = 'trained_ae/test_ae.pt'
target = 'full'
channel_label = 'Electrode'
channels_out = '4'
timeseries_out = '25'
n_epochs = '10'

In [5]:
!python .\autoencoder_training_main.py path_dataset={path_dataset} save_name={save_name} target={target} channel_label={channel_label} channels_out={channels_out} timeseries_out={timeseries_out} n_epochs={n_epochs}"

^C
